# Regresión logistica

Ayuda a clasificar los registros en categorias. Sirve para variables discretas.

Predecir si un cliente aleatorio que entra en la tienda va a comprar un producto en particular basandonos en sus ingresos, géneros, historial de compras, etc.

**Regresión lineal**

\begin{equation}
y = \alpha + \beta x
\end{equation}

![|0.1](../img/Linear_regression.svg.png)

**Regresión logística**

\begin{equation}
y = \frac{1}{1+e^{-(\alpha+\beta x)}}
\end{equation}

![](../img/Funcion-Logistica.jpg)

## matemáticas tras la regresión logística
#### Tablas de contingencia

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../datasets/gender-purchase/Gender Purchase.csv')
df.head()

,Gender,Purchase
0,Female,Yes
1,Female,Yes
2,Female,No
3,Male,No
4,Male,Yes


In [3]:
df.shape

(511, 2)

Ahora, utilizaremos las tablas de contingencia para evidenciar las frecuencias de repetición de alguna convinación de estas categorias. Utilizaremos el método **crosstab** para este proposito.

In [13]:
tabla_contingencia = pd.crosstab(df['Gender'], df['Purchase'])
tabla_contingencia

Purchase,No,Yes
Gender,,
Female,106,159
Male,125,121


Ahora, podemos totalizar cada una de las categorias de añadiendo una columna sobre el objeto que se creó previamente.

Podemos adicionalmente, transformar la tabla de contingencias para que nos proporciones el porcentaje de participacion de cada una de las frecuencias acumuladas, con respecto al total. Incialmente, debemos utilizar el método **astype** para trabajar con valores de tipo florante, y con el método div, realizar la división de la categoría sobre el total.

In [14]:
tabla_contingencia.astype('float').div(tabla_contingencia.sum(axis=1), axis=0)

Purchase,No,Yes
Gender,,
Female,0.40000,0.60000
Male,0.50813,0.49187


## Probabilidad condicional

Define la probabilidad de ocurrencia de un evento aleatorio, dadas unas condiciones previas conocidas con certeza de su ocurrencia o no ocurrencia.

* Cuál es la probabilidad de que un cliente compre un producto si es hombre?
* Cuál es la probabilidd de que saviendo que un cliente compra un producto, esta sea mujer?

In [15]:
from IPython.display import display, Math, Latex

**Primera pregunta de investigación**

$$P(compra|hombre) = \frac{Numero~ total~ de~ compras~ hechas~ por~ hombres~}{Numero~ total~ de~ hombres~ de~ la~ muestra~} = \frac{comprar \cap hombre}{hombre}$$


In [17]:
121/246

0.491869918699187

**Segunda pregunta de investigación**

$$P(mujer|compra) = \frac{Numero~ total~ de~ compras~ hechas~ por~ mujeres~}{Numero~ total~ de~ compras} = \frac{mujer \cap compra}{compra}$$

In [18]:
159/280

0.5678571428571428

## Ratio de probabilidades
Cociente entre los casos de éxito sobre los de fracaso en el suceso estudiado para cada grupo

$$ P_m = probabilidad ~de ~compras ~hechas ~por ~hombres$$
$$ P_f = probabilidad ~de ~compras ~hechas ~por ~mujeres$$
$$ odds_{compre|hombre} = \frac{P_m}{1-P_m}$$
$$ odds_{compre|hombre} = \frac{P_f}{1-P_f}$$

In [21]:
pm = 121/246
pf = 159/265

odds_m = pm/(1-pm)
odds_f = pf/(1-pf)
print(odds_f, odds_m)

1.4999999999999998 0.9680000000000002


# Implementación del método de la máxima verosimilitud para la regresión lineal

## Defir la función de entorno L(b)

$$ L(\beta) = \sum_{i=1}^n P_i^{y_i} (1-P_i)^{y_i} $$

In [36]:
def funcionEntorno(y, pi):
    import numpy as np
    total_sum = 1
    sum_in = list(range(1, len(i+1)))

    for i in range(len(y)):
        sum_in[i] = np.where(y[i] == i, pi[i], 1-pi[i])
        total_sum = total_sum*sum_in[i]
    return total_sum

### Calcular las probabilidades para cada observación

$$ P_i = P(x_i) = \frac{1}{1 + e^{-\beta\cdot x_i}}$$

In [34]:
def probLogisticas(X, beta):
    import numpy as np
    n_row = np.shape(X)[0]
    n_col = np.shape(X)[1]
    pi = list(range(1, n_row+1))
    expon = list(range(1, n_row+1))
    for i in range(n_row):
        expon[i] = 0
        for j in range(n_col):
            ex = X[i][j] * beta[j]
            expon[i] = ex + expon[i]
        with np.errstate(divide="ignore", invalid="ignore"):
            pi[i] = 1/(1+np.exp(-expon[i]))
    return pi

### Calcular la matriz diagonal W

$$ W = diag(P_i \cdot (1-P_i))_{i=1}^n$$

In [24]:
def findW(pi):
    import numpy as np
    n = len(pi)

    W = np.zeros(n*n).reshape(n,n)
    for i in range(n):
        print(i)
        W[i,i] = pi[i]*(1-pi[i])
        W[i,i].astype('float')
    return W


### Obtener la solución de la función logistica

#### metodo de newton rapson

$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$

In [25]:
def logistic(X, Y, limit):
    import numpy as np
    from numpy import linalg

    n_row = np.shape(X)[0]
    bias = np.ones(n_row).reshape(n_row, 1)
    X_new = np.append(X, bias, axis=1)
    ncol = np.shape(X_new)[1]
    beta = np.zeros(ncol).reshape(ncol, 1)
    root_dif = np.array(range(1, ncol+1)).reshape(ncol, 1)
    iter_i = 10000

    while (iter_i > limit):
        print(str(iter_i)+","+str(limit))
        pi = probLogisticas(X_new, beta)
        print(pi)
        W = findW(pi)
        print(W)
        numerato = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose())
        denominator = np.matrix(np.transpose(X_new)*np.matrix(W)*np.matrix(X_new))
        root_dif = np.array(linalg.inv(denominator)*numerato)
        beta = beta + root_dif
        print(beta)
        iter_i = np.sum(root_dif*root_dif)
        print(iter_i)
        ll = funcionEntorno(Y, pi)
    return beta


## comprobacion experimental

In [27]:
import numpy as np


In [28]:
X = np.array(range(10)).reshape(10,1)
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [29]:
Y = [0, 0, 0, 0, 1, 0, 1,0,1,1]

In [30]:
bais = np.ones(10).reshape(10,1)
X_new = np.append(X, bais, axis=1)
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [37]:
a = logistic(X, Y, 0.00001)

10000,1e-05
[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
[[ 0.43636364]
 [-2.36363636]]
5.777190082644626


UnboundLocalError: cannot access local variable 'i' where it is not associated with a value